In [2]:
import numpy as np
import pandas as pd
import random 

In [3]:
def generate_gender_critiera(docset):
    male_terms = ["man", "men", "male", "boy", "boys"]
    female_terms = ["woman", "women", "female", "girl", "girls", "pregnant", "menopausal", "postmenopausal"]
    
    # need conditional logic for {male; female; both} and termset
    

In [72]:
def generate_age_critiera(docset, min_age=0, max_age=100, comparator=">=", target_value=18):
    
    age_nlpql = """
    termset AgeTerms:
       ["age","aged"];

    define Age:
       Clarity.ValueExtraction({
         termset:[AgeTerms],
         documentset: [%s],
         minimum_value: "%d",
         maximum_value: "%d"
         });

    define final meetsAgeCriteria:
        where Age.value %s %d;""" % (docset, min_age, max_age, comparator, target_value)
    
    return age_nlpql

In [73]:
def generate_race_critiera(docset):
    
    race_nlpql =  """define RaceFinderFunction:
    Clarity.RaceFinderTask({
        documentset: [%s]
    })""" % docset
    
    return race_nlpql


In [74]:
def generate_location_critiera(docset):
    pass

In [75]:
def generate_medication_criteria(docset):
    pass

In [76]:
def generate_condition_criteria(docset):
    pass

In [77]:
# need a way to gracefully handle empty strings OR paramterize the list of termsets we actually evaluate
gender_criteria = """termset Gender: [];"""
age_criteria =  """termset Age: [];"""
location_criteria =   """termset Location: [];"""
conditions_criteria =  """termset Condtions: [];"""
medications_criteria =   """termset Medications: [];"""

docset_name = "PotentiallyEligiblePatientsNotes"

synthetic_query_template ="""

// Phenotype library name
phenotype "Synthetic Query" version "1";

/* Phenotype library description */
description "Synthetically generated, paramaterized query; for computing benchark runtime stats";

// # Structured Data Model #
datamodel OMOP version "5.3";

// # Referenced libraries #
// The ClarityCore library provides common functions for simplifying NLP pipeline creation
include ClarityCore version "1.0" called Clarity;
include OHDSIHelpers version "1.0" called OHDSI;

// ## Code Systems ##
codesystem OMOP: "http://omop.org"; // OMOP vocabulary https://github.com/OHDSI/Vocabulary-v5.0;


// #Manual Term sets#
// simple example-- termset "Vegetables":["brocolli","carrots","cauliflower"]
// can add expansion of structured concepts from terminologies as well with OMOPHelpers

documentset %s:
    Clarity.createReportTagList(["Physician","Nurse","Note","Discharge Summary"]); 

// Gender inclusion criteria termset, if gender criteria present
%s

// Age inclusion value extraction task, if age criteria present
%s

// Location inclusion criteria termset, if location criteria present
%s

// Condition(s) inclusion criteria termset, if condition criteria present
%s

// Medication(s) inclusion criteria termset, if medication criteria present
%s


define final isEligiblePatient:
  Clarity.ProviderAssertion({
    termset: [], #fix this
    documentset: [] #fix this
});

""" % (docset_name, 
       gender_criteria,
       generate_age_critiera(docset_name),
       location_criteria, 
       conditions_criteria, 
       medications_criteria, 
       generate_race_critiera(docset_name),
       docset_name) 


TypeError: not all arguments converted during string formatting